Two plastic sheets, each 5 mm thick, are to be bonded together with a thin layer of adhesive which fuses at 140$^\circ$C. For this purpose, they are pressed between two surfaces at 250$^\circ$C (See figure 5). Determine the time for which the two sheets should be pressed together, if the initial temperature of the sheets (and the adhesive) is 30$^\circ$C. For plastic sheets, 
    $k = 0.26 \, \text{W/m-K}, \quad C = 2000 \, \text{J/kg-K}, \quad \rho = 1300 \, \text{kg/m}^3.$

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# =============================== CONFIG ===============================
# Geometry
L_total = 0.010          # [m] slab thickness (two 5 mm sheets)
Nx = 201                 # number of nodes across thickness

# Material properties
k, rho, cp = 0.26, 1300.0, 2000.0   # [W/m-K], [kg/m^3], [J/kg-K]

# Boundary & initial conditions
T_left, T_right = 250.0, 250.0   # [°C] fixed face temps
T_initial = 30.0                 # [°C] initial temperature
T_fusion = 140.0                 # [°C] adhesive fusion temp

# Time control
safety = 0.45    # stability factor (<=0.5 for explicit)
total_time = 200.0  # [s] max allowed simulation time

# Animation controls
frame_stride = 500   # save every Nth time step
repeat_rows = 50     # visual extrusion in y
colormap_name = "plasma"
# =====================================================================

alpha = k/(rho*cp)
dx = L_total/(Nx-1)
dt_max = dx**2/(2*alpha)
dt = safety*dt_max
n_steps = int(np.ceil(total_time/dt))

x = np.linspace(0.0, L_total, Nx)
T = np.full(Nx, T_initial, dtype=float)
T[0], T[-1] = T_left, T_right
mid_idx = (Nx-1)//2

# Storage for snapshots
snap, times, mids = [T.copy()], [0.0], [T[mid_idx]]

# Time stepping
fuse_time = None
for n in range(1, n_steps+1):
    Tn = T.copy()
    Tn[1:-1] = T[1:-1] + alpha*dt*(T[2:] - 2*T[1:-1] + T[:-2]) / dx**2
    Tn[0], Tn[-1] = T_left, T_right
    T = Tn
    
    if n % frame_stride == 0 or n == n_steps:
        snap.append(T.copy())
        times.append(n*dt)
        mids.append(T[mid_idx])
    
    if T[mid_idx] >= T_fusion:   # stop if adhesive melts
        fuse_time = n*dt
        snap.append(T.copy())
        times.append(n*dt)
        mids.append(T[mid_idx])
        break



In [5]:
# ------------------------- Inline Animation --------------------------
fig, ax = plt.subplots(figsize=(7.2, 3.8))
img = np.tile(snap[0], (repeat_rows, 1))
im = ax.imshow(img, aspect="auto", origin="lower",
               extent=[0, L_total*1000.0, 0, 1],
               vmin=min(T_initial, T_left, T_right),
               vmax=max(T_left, T_right),
               cmap=colormap_name)
cb = plt.colorbar(im, ax=ax)
cb.set_label("Temperature [°C]")

ax.set_xlabel("Position across thickness [mm]")
ax.set_yticks([])
title_txt = ax.set_title("Heating of Slab — Colormap View")
time_txt = ax.text(0.02, 0.95, "", transform=ax.transAxes, va="top")
mid_txt  = ax.text(0.02, 0.88, "", transform=ax.transAxes, va="top", color="yellow")

def init():
    im.set_data(np.tile(snap[0], (repeat_rows, 1)))
    time_txt.set_text(f"t = {times[0]:.2f} s")
    mid_txt.set_text(f"Center: {mids[0]:.1f} °C")
    return im, time_txt, mid_txt

def update(i):
    im.set_data(np.tile(snap[i], (repeat_rows, 1)))
    time_txt.set_text(f"t = {times[i]:.2f} s")
    mid_txt.set_text(f"Center: {mids[i]:.1f} °C")
    return im, time_txt, mid_txt

ani = FuncAnimation(fig, update, frames=len(snap),
                    init_func=init, blit=True, interval=100)
plt.close(fig)  # avoid duplicate static image

HTML(ani.to_jshtml())
